# Notebook Test for Project

> A test for emotion classification of eds version

In [ ]:
# | default_exp core

In [ ]:
# | hide

openface_url = 'http://192.168.16.153:8002/openface/'
emotion_url = "http://192.168.16.162:8010/YXCSI_ALG_Server/v1"

# the sample is from http://git.xinktech.com/yscz/alg/hr_detect_server/uploads/81dac5cbf6904113c61dee9305e9eada/sample.mp4
video_files = ['/home/wangli/projects/yxcsi_alg_server/data/wrong_anger.mp4',
              '/home/wangli/projects/yxcsi_alg_server/data/wrong_fear.mp4']
#video_files = ['/home/wangli/data/six_emotions/Disgust/F06-Disgust-FaceForward.avi']
emotion_classes = ['Anger', 'Contempt', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']

In [ ]:
# | hide
import base64
import time
import json
import cv2
import requests
import plotly.graph_objects as go
import numpy as np

In [ ]:
def convert_jpg_base64(image):

    # convert to jpg code
    ret, framej = cv2.imencode('.jpg', image)
    # Encode the bytes-like object s using Base64 and return the encoded bytes.
    framej = base64.b64encode(framej)
    # decode the bytes from base64.b64encode as a UTF-8 string
    framej = framej.decode('utf-8')

    return framej


def openface_request(image_base64, openface_url):
    '''
    return: openface接口返回数据
    '''
    #openface_url = 'http://127.0.0.1:8002/openface'
    datas = json.dumps({'image': image_base64})
    response = requests.post(openface_url, data=datas)
    return response

In [ ]:
def classify_emotion_request(frame):
    framej = convert_jpg_base64(frame)
    datas = json.dumps({'image': framej, "imageid": 111, "timestamp": 112345566})

    response = requests.post(emotion_url, data=datas)
    if response.status_code != 200:
        return None
    if response.json() == {}:
        return None
    return response.json()['microexpressions']

In [ ]:
def detect_emotion_from_video(video_path, detect_func):

    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print('frame_count: ', frame_count)
    print('fps: ', fps)

    # extract frames
    frame_emotions = []
    count = 0

    while True:
        ret, frame = cap.read()
        if ret:

            count += 1
        else:
            break
        # convert to rgb
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # detect emotion
        emotion = detect_func(frame)
        if emotion is None:
            continue
        frame_emotions.append(emotion)
        print('frame: ', count, 'emotion: ', emotion)

    cap.release()
    return frame_emotions


def plot_lines(frame_emotions):

    frame_emotions = np.array(frame_emotions)
    
    fig = go.Figure()
    for i in range(len(emotion_classes)):
        fig.add_trace(go.Scatter(x=list(range(frame_emotions.shape[0])),
                                 y=frame_emotions[:,i].tolist(),
                                 mode='lines',
                                 name=emotion_classes[i]))

    # 设置图表标题和轴标签
    fig.update_layout(title="Signal Curves", xaxis_title="X-axis", yaxis_title="Y-axis")

    # 显示图表
    fig.show()

In [ ]:
for video in video_files:
    frame_emotions = detect_emotion_from_video(video, classify_emotion_request)
    plot_lines(frame_emotions)

frame_count:  118
fps:  30
frame:  15 emotion:  [0.06, 0.02, 0.02, 0.08, 0.23, 0.17, 0.05, 0.37]
frame:  16 emotion:  [0.05, 0.02, 0.02, 0.06, 0.17, 0.35, 0.11, 0.21]
frame:  17 emotion:  [0.06, 0.02, 0.04, 0.08, 0.1, 0.37, 0.08, 0.25]
frame:  18 emotion:  [0.08, 0.02, 0.05, 0.08, 0.11, 0.3, 0.08, 0.28]
frame:  19 emotion:  [0.07, 0.02, 0.06, 0.09, 0.16, 0.29, 0.08, 0.24]
frame:  20 emotion:  [0.05, 0.02, 0.04, 0.07, 0.24, 0.26, 0.07, 0.25]
frame:  21 emotion:  [0.06, 0.02, 0.03, 0.08, 0.23, 0.26, 0.08, 0.24]
frame:  22 emotion:  [0.06, 0.02, 0.02, 0.07, 0.23, 0.29, 0.09, 0.22]
frame:  23 emotion:  [0.05, 0.02, 0.02, 0.07, 0.26, 0.26, 0.07, 0.24]
frame:  24 emotion:  [0.05, 0.03, 0.02, 0.06, 0.27, 0.29, 0.07, 0.22]
frame:  25 emotion:  [0.05, 0.02, 0.02, 0.05, 0.29, 0.28, 0.07, 0.23]
frame:  26 emotion:  [0.06, 0.02, 0.01, 0.07, 0.26, 0.25, 0.07, 0.26]
frame:  27 emotion:  [0.07, 0.02, 0.01, 0.08, 0.2, 0.28, 0.07, 0.27]
frame:  28 emotion:  [0.06, 0.01, 0.01, 0.09, 0.23, 0.27, 0.05, 0.

frame_count:  915
fps:  29
frame:  8 emotion:  [0.07, 0.06, 0.07, 0.11, 0.26, 0.22, 0.1, 0.12]
frame:  9 emotion:  [0.06, 0.06, 0.06, 0.11, 0.24, 0.23, 0.09, 0.13]
frame:  10 emotion:  [0.06, 0.06, 0.06, 0.14, 0.24, 0.21, 0.07, 0.16]
frame:  12 emotion:  [0.06, 0.07, 0.08, 0.13, 0.2, 0.23, 0.08, 0.16]
frame:  13 emotion:  [0.05, 0.08, 0.07, 0.12, 0.21, 0.23, 0.07, 0.18]
frame:  14 emotion:  [0.06, 0.06, 0.06, 0.15, 0.2, 0.21, 0.08, 0.18]
frame:  15 emotion:  [0.05, 0.07, 0.05, 0.15, 0.2, 0.23, 0.07, 0.19]
frame:  16 emotion:  [0.04, 0.06, 0.04, 0.13, 0.22, 0.24, 0.07, 0.18]
frame:  17 emotion:  [0.04, 0.06, 0.04, 0.16, 0.21, 0.23, 0.08, 0.19]
frame:  18 emotion:  [0.04, 0.07, 0.05, 0.15, 0.21, 0.22, 0.08, 0.18]
frame:  19 emotion:  [0.05, 0.07, 0.05, 0.14, 0.18, 0.24, 0.07, 0.19]
frame:  20 emotion:  [0.05, 0.06, 0.05, 0.13, 0.19, 0.25, 0.06, 0.21]
frame:  21 emotion:  [0.04, 0.07, 0.05, 0.12, 0.21, 0.26, 0.05, 0.19]
frame:  22 emotion:  [0.04, 0.09, 0.05, 0.11, 0.21, 0.25, 0.06, 0.2]


In [ ]:
plot_lines(frame_emotions)